## Load In Libraries

In [1]:
import math
import ctREFPROP.ctREFPROP as ct
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Hack for my installation
os.environ['RPPREFIX'] = r'C:\\Program Files (x86)\REFPROP'

root = os.environ['RPPREFIX']
#r = ct.REFPROPFunctionLibrary(os.path.join(root, 'refprop.dll'))
r = ct.REFPROPFunctionLibrary(os.path.join(root, 'REFPRP64.dll'))
r.SETPATHdll(root)

## Real Gas Data Generation

In [2]:
fuel_type = 'ethane.FLD'
amb_type='nitrogen.FLD'

## Conditions
Pchamb = 2000
Pinj= Pchamb#013700##########make Pinj=Pchamb for isothermal assumption
Tamb = 305.33*np.array([0.6,0.8,1,1.3])
TfuelIn = 266.02


## Construct fluid mixture
mix_list=fuel_type+"|"+amb_type
ierr, herr = r.GERG08dll(2,0)
if ierr !=0:
    raise ValueError(str(ierr) + ':' + herr)

r.FLAGSdll('GERG',1)
ierr, herr = r.SETUPdll(2,mix_list,'HMX.BNC','DEF')

if ierr !=0:
    raise ValueError(str(ierr) + ':' + herr)
zf=[1,0]
zamb=[0,1]


    
## Initialize
dX = 0.01
xf = np.arange(0,1+dX,dX)
yf = np.zeros(xf.size)
rhomix = np.zeros(xf.size)
Tmix = np.zeros((xf.size,Tamb.size))
Rmix = np.zeros(xf.size)
nmix = np.zeros(xf.size)
delmix = np.zeros(xf.size)
phi = np.zeros(xf.size)
MW_mix = np.zeros(xf.size)
T_sat_V = np.zeros(xf.size)
T_sat_L = np.zeros(xf.size)
## Sat values
i=-1
for x in xf:
    i+=1
    z=[x,(1-x)]
    satprops = r.SATPdll(Pchamb,z,2)
    T_sat_V[i] = satprops.T
    satprops = r.SATPdll(Pchamb,z,1)
    T_sat_L[i] = satprops.T

jj=-1
total_steps = Tamb.size
for T in Tamb:
    jj+=1
    fInlet = r.TPFLSHdll(TfuelIn,Pinj,zf)
    Amb = r.TPFLSHdll(T,Pchamb,zamb)
    HfInlet = fInlet.h
    HAmb = Amb.h
 
    i=-1
    for x in xf:
        i+=1
        hmix = HfInlet*x+HAmb*(1-x)
        z=[x,(1-x)]
        MW_mix[i] = r.WMOLdll(z)
        Props = r.PHFLSHdll(Pchamb,hmix,z)
        rhomix[i] = Props.D*MW_mix[i]
        Tmix[i,jj] = Props.T
        if Props.ierr!=0:
            Tmix[i,jj] = None
    print('%d / %d'%(jj+1, total_steps),end='\r')

In [3]:
i=-1
for T in T_sat_V:
    i+=1
    if T_sat_V[i]>350:
        T_sat_V[i]=None
    if T_sat_L[i]>350:
        T_sat_L[i]=None 
T_sat_L[0:50]=None


a = pd.Series(T_sat_V)
i=-1
while i<1:
    i+=1
    #aint = a.interpolate(method="pad", limit=2)
    aint = a.interpolate(method="polynomial", order=3)
    #aint = a.interpolate(method="linear")
    T_sat_V2 = aint.to_numpy()
np.count_nonzero(np.isnan(T_sat_V2))

a = pd.Series(T_sat_L)
i=-1
while i<1:
    i+=1
    #aint = a.interpolate(method="pad", limit=2)
    aint = a.interpolate(method="polynomial", order=3)
    #aint = a.interpolate(method="linear")
    T_sat_L2 = aint.to_numpy()
np.count_nonzero(np.isnan(T_sat_L2))


Tmix3 = np.zeros((xf.size,Tamb.size))
Tmix2 = np.zeros((xf.size,Tamb.size))
jj=-1
for Tsweep in Tamb:
    jj+=1
    TsingleMix = Tmix[:,jj]
    a = pd.Series(TsingleMix)
    i=-1
    while i<1:
        i+=1
        #aint = a.interpolate(method="pad", limit=2)
        aint = a.interpolate(method="polynomial", order=3)
        #aint = a.interpolate(method="linear")
        Tmix2[:,jj] = aint.to_numpy()

    i=-1
    for T in T_sat_V2:
        i+=1
        if Tmix2[i,jj]<T:
            Tmix3[i,jj]=None
        else:
            Tmix3[i,jj]=Tmix2[i,jj]

## Tmix Plot

In [26]:
import matplotlib as mpl

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIGGER_SIZE = 20

fig, ax = plt.subplots(constrained_layout=True)
#HSV= plt.get_cmap('cool')
#colors = iter(HSV(np.linspace(0,1,TfuelIn.size)))
colors = ['y','r','c','m','r','c']
jj=-1
for T in Tamb:
    jj+=1
    #color=next(colors)
    color = colors[jj]
    plt.plot(xf,Tmix3[:,jj],color,linewidth=4)
plt.plot(xf,T_sat_V2,'k',linewidth=4)
plt.plot(xf,T_sat_L2,'k',linewidth=4)
plt.plot(xf[90:100],Tmix[90:100,0],'y',linewidth=4)
plt.plot(xf[-1],T_sat_V2[-1],'ko',markersize=8)
jj=-1
for T in Tamb:
    jj+=1
    color=colors[jj]
    if jj==0:
        plt.plot(xf,Tmix2[:,jj],'y--',linewidth=4)
    if jj==1:
        plt.plot(xf,Tmix2[:,jj],'k--',linewidth=4)
    plt.plot(xf,Tmix3[:,jj],color,linewidth=4)
    
plt.xticks(fontsize=BIGGER_SIZE)
plt.yticks(fontsize=BIGGER_SIZE)
plt.xlabel('Ethane Mole Fraction [-]')
plt.ylabel('Temperature [K]')
#plt.legend(['Boiling Line','Cricondentherm'])
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
font = {'family' : 'normal',
        'weight' : 'normal'}
ax.tick_params(direction='out', length=6, width=2, colors='k',
               grid_color='r', grid_alpha=0.5,right='bool',top='bool')
plt.rc('font', **font)
#plt.text(0.2,287,'Point',fontsize=SMALL_SIZE, color='k')
plt.text(0.4,200,'Vapor-Liquid Equilibrium',fontsize=14, color='k')
plt.text(0.1,337,'T$_{N_2}$= 1.3 T$_{c,f}$',fontsize=14, color='m',rotation=-30)
plt.text(0.1,290,'T$_{N_2}$= T$_{c,f}$',fontsize=14, color='c',rotation=-15)
plt.text(0.1,253,'T$_{N_2}$= 0.8 T$_{c,f}$',fontsize=14, color='r',rotation=0)
#plt.text(0.1,216,'T$_{N2}$= 0.65 T$_{c}$',fontsize=14, color='g',rotation=10)
plt.text(0.1,180,'T$_{N_2}$= 0.6 T$_{c,f}$',fontsize=14, color='y',rotation=15)
ax.set_yticks(np.linspace(100, 400, 7))
plt.ylim([150, 400])
plt.xlim([0, 1])
plt.show()



ValueError: 
$ho$
^
Expected end of text, found '$'  (at char 0), (line:1, col:1)

<Figure size 640x480 with 1 Axes>

In [24]:
image_format = 'png' # e.g .png, .svg, etc.
image_name = 'VaporMixing.png'

fig.savefig(image_name, format=image_format, dpi=1200)

## Ideal Gas Data Gen

In [21]:

Ru = 8.314

## Initialize arrays
Tstart = 300
dT = 1
Tsweep = np.arange(Tstart-100,Tamb+dT+50,dT)
HTotal = np.zeros(xf.size)
TPure = np.zeros(xf.size)
TIdeal = np.zeros(xf.size)
TotalPure = np.zeros(xf.size)
rhoPure = np.zeros(xf.size)
rhoIdeal = np.zeros(xf.size)
nIdeal = np.zeros(xf.size)
nPure = np.zeros(xf.size)
delIdeal = np.zeros(xf.size)
delPure = np.zeros(xf.size)


## Intialize chamber inlet properties
yf =((xf*MWf)/(xf*MWf+(1-xf)*MWamb))
if CaseAmb==1:
    amb_props1=r.TPFLSHdll(Tamb,Pchamb,[0,1,0,0])
    amb_props2=r.TPFLSHdll(Tamb,Pchamb,[0,0,1,0])
    amb_props3=r.TPFLSHdll(Tamb,Pchamb,[0,0,0,1])
    rhoamb = amb_props1.D*0.78+amb_props2.D*0.21+amb_props3.D*0.01
    HAmb = amb_props1.h*0.78+amb_props2.h*0.21+amb_props3.h*0.01
    
if CaseAmb==2:
    amb_props1=r.TPFLSHdll(Tamb,Pchamb,[0,1])
    rhoamb = amb_props1.D
    HAmb = amb_props1.h
    
fuel_in_props=r.TPFLSHdll(TfuelIn,Pinj,zf)
HfInlet = fuel_in_props.h 
fuel_chamber_props=r.PHFLSHdll(Pchamb,HfInlet,zf)
total_steps_xf=xf.size
total_steps_T=Tsweep.size
i=-1
for x in xf:
    i+=1
    HTotal[i] = HfInlet*x+HAmb*(1-x) 
## NIST thermophysical properties
hfArray = np.zeros(Tsweep.size)
hAmb = np.zeros(Tsweep.size)
hfArrayIdeal = np.zeros(Tsweep.size)
hambArrayIdeal = np.zeros(Tsweep.size)
dH = 10
hsweep=np.arange(HfInlet,HAmb+dH,dH)
DfArray = np.zeros(hsweep.size)
Damb1Array = np.zeros(hsweep.size)
Damb2Array = np.zeros(hsweep.size)
Damb3Array = np.zeros(hsweep.size)
i=-1
for x in xf:
    if x > xflast:
        break
    i+=1
    j=-1
    for T in Tsweep:
        j+=1
        ##Pure Props
        fuelProps = r.TPFLSHdll(T,Pchamb,zf)
        if CaseAmb==1:
            amb1props = r.TPFLSHdll(T,Pchamb,[0,1,0,0])
            amb2props = r.TPFLSHdll(T,Pchamb,[0,0,1,0])
            amb3props = r.TPFLSHdll(T,Pchamb,[0,0,0,1])
            hamb1Array = amb1props.h
            hamb2Array = amb2props.h
            hamb3Array = amb3props.h
            hAmb[j] = hamb1Array*0.78+hamb2Array*0.21+hamb3Array*0.01
        if CaseAmb==2:
            amb1props = r.TPFLSHdll(T,Pchamb,[0,1])
            hamb1Array = amb1props.h
            hAmb[j] = hamb1Array
        hfArray[j] = fuelProps.h 
        

        ##Ideal Props
        fuelPropsIdeal = r.TPFLSHdll(T,0,zf)
        ambpropsIdeal = r.TPFLSHdll(T,0,zamb)
        cpf = fuelPropsIdeal.Cp
        cpamb = ambpropsIdeal.Cp
        hfArrayIdeal[j] = fuelPropsIdeal.h #= cpf*T
        hambArrayIdeal[j] = ambpropsIdeal.h #cpamb*T
    check=1
    checkIdeal=1
    Value=0
    ValueIdeal=0
     
    hmix = hfArray*x+hAmb*(1-x) 
    hmixIdeal = hfArrayIdeal*x+hambArrayIdeal*(1-x)
    while check==1:
        Value+=10
        pp=-1
        for kk in hmix:
            pp+=1
            if kk<(HTotal[i]+Value) and kk>(HTotal[i]-Value):
                TPure[i] = Tsweep[pp]
                check=0
    while checkIdeal==1:
        ValueIdeal+=10
        pp=-1
        for kk in hmixIdeal:
            pp+=1
            if kk<(HTotal[i]+ValueIdeal) and kk>(HTotal[i]-ValueIdeal):
                TIdeal[i] = Tsweep[pp]
                checkIdeal=0
    ## Density
    fuelProps=r.TPFLSHdll(TPure[i],Pchamb,zf)
    if CaseAmb==1:
        amb1props=r.TPFLSHdll(TPure[i],Pchamb,[0,1,0,0])
        amb2props=r.TPFLSHdll(TPure[i],Pchamb,[0,0,1,0])
        amb3props=r.TPFLSHdll(TPure[i],Pchamb,[0,0,0,1])
        amb1D = amb1props.D
        amb2D = amb2props.D
        amb3D = amb3props.D
        DAmb = amb1D*0.78+amb2D*0.21+amb3D*0.01 
    if CaseAmb==2:
        amb1props=r.TPFLSHdll(TPure[i],Pchamb,[0,1])
        amb1D = amb1props.D
        DAmb = amb1D
    TotalPure[i]=(rF*fuelProps.D*MWf*x+rAmb*DAmb*MWamb*(1-x))
    rhoPure[i] = fuelProps.D*x*MWf+DAmb*(1-x)*MWamb
    rhoIdeal[i] = Pchamb/(Ru*TIdeal[i])*MW_mix[i]
    nIdeal[i] = math.sqrt((2*Rmix[i]*rhoIdeal[i]+1)/(1-Rmix[i]*rhoIdeal[i]))
    nPure[i] = math.sqrt((2*TotalPure[i]+1)/(1-TotalPure[i]))
    delIdeal[i]=nIdeal[i]/nIdeal[0]-1
    delPure[i]=nPure[i]/nPure[0]-1
    print('%d / %d'%(i, total_steps_xf),end='\r')


NameError: name 'Pamb' is not defined

In [130]:
from numpy import loadtxt
import logging
import sys
if '-c' in sys.argv:
    tecplot.session.connect()
import tecplot
import numpy as np
import tecplot as tp
from tecplot.constant import *
from tecplot.constant import PlotType, AxisMode, AxisAlignment, TickDirection

#Setup Tecplot Frame
logging.basicConfig(level=logging.DEBUG)
tecplot.new_layout()
frame = tp.active_frame()

#Enter Data
xlabel2 = 'Fuel Mole Fraction'
ylabel1 = '<greek> f </greek>'
#ylabel1 = 'Mixture Fraction'
ylabel2 = 'Tmix'
x1 = delmix
x2 = xf
y1 = phi
y2 = Tmix
dataset = frame.create_dataset('Data', [xlabel1, ylabel1,ylabel2,xlabel2])
zone = dataset.add_ordered_zone('Real', (x1.size,))
zone.values(xlabel1)[:] = x1
zone.values(ylabel1)[:] = y1
zone.values(ylabel2)[:] = y2
zone.values(xlabel2)[:] = x2

x1 = delIdeal
y2 = TIdeal

zone = dataset.add_ordered_zone('Ideal', (x1.size,))
zone.values(xlabel1)[:] = x1
zone.values(ylabel1)[:] = y1
zone.values(ylabel2)[:] = y2
zone.values(xlabel2)[:] = x2

x1 = delPure
y2 = TPure
zone = dataset.add_ordered_zone('Pure', (x1.size,))
zone.values(xlabel1)[:] = x1
zone.values(ylabel1)[:] = y1
zone.values(ylabel2)[:] = y2
zone.values(xlabel2)[:] = x2


plot = frame.plot(PlotType.XYLine)
plot.activate()
for lmap in plot.linemaps():
    lmap.show = True
    lmap.line.line_thickness = 0.6
yaxis = plot.axes.y_axis(0)
yticks = yaxis.ticks
yticks.direction = TickDirection.In
plot.axes.precise_grid.show = True
plot.axes.x_axis(0).grid_lines.show = True
#plot.axes.axis_mode = AxisMode.XYDependent
#lmap.y_axis.title.color = Color.DeepRed

# Outputs .png file
tecplot.save_layout('delVsTmixVsPhi.lay',include_data=True)


In [3]:
total_steps=xf.size
f = open("Optical2ThermoTableAllen",'w')    


kk=0
print('starting file writing...')




for i in xf:
    f.write('%14.7e %14.7e %14.7e %14.7e %14.7e \n'%(delmix[kk],xf[kk],Tmix[kk],phi[kk],yf[kk]))
    kk+=1
    print('%.3f'%(kk/total_steps*100),end='%\r')
f.close()

starting file writing...


In [60]:
from numpy import loadtxt
import logging
import sys
if '-c' in sys.argv:
    tecplot.session.connect()
import tecplot
import numpy as np
import tecplot as tp
from tecplot.constant import *
from tecplot.constant import PlotType, AxisMode, AxisAlignment, TickDirection

#Setup Tecplot Frame
logging.basicConfig(level=logging.DEBUG)
tecplot.new_layout()
frame = tp.active_frame()

#Enter Data
xlabel1 = 'Fuel Mole Fraction1'
xlabel2 = 'Fuel Mole Fraction2'
xlabel3 = 'Fuel Mole Fraction3'
ylabel1 = 'Tmix'
ylabel2 = 'TsatV'
ylabel3 = 'TsatL'




x1 = xf
y1 = Tmix
y2 = T_sat_V
y3 = T_sat_L


dataset = frame.create_dataset('Data', [xlabel1, ylabel1,xlabel2,ylabel2,xlabel3,ylabel3])
zone = dataset.add_ordered_zone('Tmix', (x1.size,))
zone.values(xlabel1)[:] = x1
zone.values(ylabel1)[:] = y1
zone.values(xlabel2)[:] = x1
zone.values(ylabel2)[:] = y2
zone.values(xlabel3)[:] = x1
zone.values(ylabel3)[:] = y3




plot = frame.plot(PlotType.XYLine)
plot.activate()
for lmap in plot.linemaps():
    lmap.show = True
    lmap.line.line_thickness = 0.6
yaxis = plot.axes.y_axis(0)
yticks = yaxis.ticks
yticks.direction = TickDirection.In
plot.axes.precise_grid.show = True
plot.axes.x_axis(0).grid_lines.show = True
#plot.axes.axis_mode = AxisMode.XYDependent
#lmap.y_axis.title.color = Color.DeepRed

# Outputs .png file

# Outputs .png file
tecplot.data.save_tecplot_plt('Trans_vs_super_30.plt',include_data=True)

Dataset(uid=381, frame=Frame(uid=372, page=Page(uid=371)))

In [21]:
 #Load Libraries
import logging
import sys
if '-c' in sys.argv:
    tecplot.session.connect()
import tecplot
import numpy as np
import tecplot as tp
from tecplot.constant import *
from tecplot.constant import PlotType, AxisMode, AxisAlignment, TickDirection

#Setup Tecplot Frame
logging.basicConfig(level=logging.DEBUG)
tecplot.new_layout()
frame = tp.active_frame()

#Enter Data
ylabel = 'Temperature'
xlabel = 'Fuel Mole Fraction'
dataset = frame.create_dataset('Data', [xlabel, ylabel])

Zone = 'Saturated Liquid'
zone = dataset.add_ordered_zone(Zone, (T_sat_L.size,))
zone.values(ylabel)[:] = T_sat_L
zone.values(xlabel)[:] = xf

Zone = 'Saturated Vapor'
zone = dataset.add_ordered_zone(Zone, (T_sat_L.size,))
zone.values(ylabel)[:] = T_sat_V
zone.values(xlabel)[:] = xf

Zone = 'Mixture Temperature'
zone = dataset.add_ordered_zone(Zone, (T_sat_L.size,))
zone.values(ylabel)[:] = Tmix
zone.values(xlabel)[:] = xf


plot = frame.plot(PlotType.XYLine)
plot.activate()
for lmap in plot.linemaps():
    lmap.show = True
    lmap.line.line_thickness = 0.6
yaxis = plot.axes.y_axis(0)
yticks = yaxis.ticks
yticks.direction = TickDirection.In
plot.axes.precise_grid.show = True
plot.axes.x_axis(0).grid_lines.show = True
#plot.axes.axis_mode = AxisMode.XYDependent
#lmap.y_axis.title.color = Color.DeepRed

# Outputs .png file
tecplot.save_layout('Isothermal_Mix.lay',include_data=True)



In [14]:
from numpy import loadtxt
import logging
import sys
if '-c' in sys.argv:
    tecplot.session.connect()
import tecplot
import numpy as np
import tecplot as tp
from tecplot.constant import *
from tecplot.constant import PlotType, AxisMode, AxisAlignment, TickDirection

#Setup Tecplot Frame
logging.basicConfig(level=logging.DEBUG)
tecplot.new_layout()
frame = tp.active_frame()

#Enter Data
xlabel1 = 'Fuel Mole Fraction1'
xlabel2 = 'Fuel Mole Fraction2'
xlabel3 = 'Fuel Mole Fraction3'
ylabel1 = 'Tmix'
ylabel2 = 'TsatV'
ylabel3 = 'TsatL'




x1 = xf
y1 = Tmix


dataset = frame.create_dataset('Data', [xlabel1, ylabel1])
zone = dataset.add_ordered_zone('Tmix', (x1.size,))
zone.values(xlabel1)[:] = x1
zone.values(ylabel1)[:] = y1





plot = frame.plot(PlotType.XYLine)
plot.activate()
for lmap in plot.linemaps():
    lmap.show = True
    lmap.line.line_thickness = 0.6
yaxis = plot.axes.y_axis(0)
yticks = yaxis.ticks
yticks.direction = TickDirection.In
plot.axes.precise_grid.show = True
plot.axes.x_axis(0).grid_lines.show = True
#plot.axes.axis_mode = AxisMode.XYDependent
#lmap.y_axis.title.color = Color.DeepRed

# Outputs .png file

# Outputs .png file
tecplot.data.save_tecplot_plt('AdiabaticMix_super_actual.plt',include_data=True)

Dataset(uid=51, frame=Frame(uid=42, page=Page(uid=41)))